In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mediapipe

# **Data Loading**

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

#MediaPipe hand detection module
mp_hands = mp.solutions.hands

# Function to extract landmarks
def extract_landmarks(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7) as hands:
        result = hands.process(img_rgb)
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                return landmarks
    return None  # If no hand is detected

# Function to create a dataset
def prepare_dataset(root_folder):
    data = []
    labels = []
    
    # Loop through folders for each letter
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isdir(folder_path) and "-samples" in folder_name:
            label = folder_name.split('-')[0].upper()  # Extract letter label
            
            # Process images in the folder
            for filename in os.listdir(folder_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    image_path = os.path.join(folder_path, filename)
                    
                    # Extract landmarks
                    landmarks = extract_landmarks(image_path)
                    if landmarks:
                        data.append(landmarks)
                        labels.append(label)
    
    # Return as a Pandas DataFrame
    df = pd.DataFrame(data)
    df['label'] = labels
    return df

# Set root folder
root_folder = "/kaggle/input/asl-alphabet-dataset/dataset"  # Root folder containing subfolders for each letter
dataset = prepare_dataset(root_folder)

print(f"Dataset size: {dataset.shape}")

# To save the data
dataset.to_csv("landmark_dataset.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Loading MediaPipe landmark dataset
asl_dataset = pd.read_csv("/kaggle/working/landmark_dataset.csv")  # CSV file of your landmark datas

# **EDA of asl-alphabet-dataset**

In [ ]:
asl_dataset.head()

In [ ]:
asl_dataset.tail()

In [ ]:
missing_values = asl_dataset.isnull().sum().sum()
print(missing_values)

In [ ]:
print("\n--- Basic Information ---")
print(f"Number of samples: {asl_dataset.shape[0]}")
print(f"Number of features (including label): {asl_dataset.shape[1]}")

In [ ]:
print("\nData Types:")
print(asl_dataset.dtypes)

In [ ]:
if 'label' in asl_dataset.columns:
    labels = asl_dataset['label']
    pixel_data = asl_dataset.drop(columns=['label'])
else:
    raise ValueError("The dataset does not contain a 'label' column.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Hypothetical label column and dataframe
# Example label column: df['label']
label_counts = asl_dataset['label'].value_counts()

# Alphabetical sorting (excluding J, from A to Z)
alphabet_labels = [chr(i) for i in range(ord('A'), ord('Z') + 1) if chr(i) != 'J']

# Adjust the graph based on label order
plt.figure(figsize=(10, 6))
sns.barplot(x=alphabet_labels, y=[label_counts.get(label, 0) for label in alphabet_labels], palette='viridis')
plt.title("Label Distribution (A–Z, No J, Sorted Alphabetically)")
plt.xlabel("Labels (A–Z, No J)")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()


In [ ]:
correlation_matrix = pixel_data.corr().abs().mean().sort_values(ascending=False)

In [ ]:
print("\n--- Correlation Analysis ---")
print(f"Mean correlation between pixel columns: {correlation_matrix.mean():.2f}")

# Visualize the correlation matrix (if the dataset is not too large)
plt.figure(figsize=(10, 8))
sns.heatmap(pixel_data.corr(), cmap="coolwarm", cbar=True)
plt.title("Correlation Matrix of Pixel Values")
plt.show()

In [ ]:
summary = {
    "Total Samples": asl_dataset.shape[0],
    "Total Features": asl_dataset.shape[1],
    "Missing Values": missing_values,
    "Unique Labels": len(label_counts) if labels is not None else "N/A",
    "Label Distribution": label_counts.to_dict() if labels is not None else "N/A",
    "Min Pixel Value": pixel_data.min(),
    "Max Pixel Value": pixel_data.max(),
    "Mean Pixel Value": pixel_data.mean(),
    "Std of Pixel Values": pixel_data.std(),
}
print("\n--- Summary of the Dataset ---")
for key, value in summary.items():
    print(f"{key}: {value}")

In [ ]:
pixel_values = pixel_data.values
# Histogram of pixel values
plt.figure(figsize=(10, 6))
plt.hist(pixel_values.flatten(), bins=50, color='skyblue')
plt.title("Pixel Value Distribution")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.show()

# **Data Preprocessing**

In [ ]:
# Separate labels and features
X_mediapipe = asl_dataset.iloc[:, :-1].values  # Landmark feature columns
y_mediapipe = asl_dataset['label'].values  # Label column

# Convert labels to numerical values and apply one-hot encoding
label_encoder = LabelEncoder()
y_mediapipe_encoded = label_encoder.fit_transform(y_mediapipe)  # Convert letters to numbers
y_mediapipe_onehot = to_categorical(y_mediapipe_encoded)  # One-hot encode

# Check the data format
print(f"Landmark Features Shape: {X_mediapipe.shape}")
print(f"Landmark Labels Shape (one-hot): {y_mediapipe_onehot.shape}")
print(f"Unique Labels: {label_encoder.classes_}")


In [ ]:
# Reshape the landmark data into a 2D tensor
X_mediapipe_reshaped = X_mediapipe.reshape(-1, 9, 7, 1)  # Example shape (9, 7, 1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convert labels to numerical values and apply one-hot encoding
label_encoder = LabelEncoder()


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_mediapipe_reshaped, y_mediapipe_onehot, test_size=0.2, random_state=42)


# **Model Training and Evaluating**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
import matplotlib.pyplot as plt


In [ ]:
def plot_training_history(history, title='Training and Validation Metrics'):
    """
    Plot training and validation accuracy and loss from the training history.

    Args:
        history (History): The training history object returned by Keras model.fit().
        title (str): The title of the plot.
    """
    # Extract training and validation metrics
    acc = history.history['accuracy']
    loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    # Create the plot
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    fig.suptitle(title, fontsize=16)

    # Accuracy plot
    ax[0].plot(epochs, acc, 'r', label='Training Accuracy')
    ax[0].plot(epochs, val_acc, 'b', label='Validation Accuracy')
    ax[0].set_title('Accuracy')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Accuracy')
    ax[0].legend()

    # Loss plot
    ax[1].plot(epochs, loss, 'r', label='Training Loss')
    ax[1].plot(epochs, val_loss, 'b', label='Validation Loss')
    ax[1].set_title('Loss')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Loss')
    ax[1].legend()

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for the title
    plt.show()


In [ ]:
def train_and_evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, batch_size=32, epochs=5):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        batch_size=batch_size, epochs=epochs, verbose=2)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    return history, test_accuracy


In [ ]:
def compute_metrics(y_true, y_pred):
    """
    Compute evaluation metrics for model predictions.
    Args:
        y_true: Ground truth labels (one-hot encoded or integer-encoded).
        y_pred: Predicted labels (probabilities or integer-encoded).

    Returns:
        A dictionary with computed metrics (macro and micro averages).
    """
    # Convert y_true (TensorFlow tensor) to a NumPy array
    if isinstance(y_true, tf.Tensor):
        y_true = y_true.numpy()

    # Convert predictions to class indices if they are probabilities
    if len(y_pred.shape) > 1:
        y_pred = y_pred.argmax(axis=1)

    # Convert ground truth from one-hot encoding to class indices
    if len(y_true.shape) > 1:
        y_true = y_true.argmax(axis=1)

    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision (Macro)': precision_score(y_true, y_pred, average='macro'),
        'Recall (Macro)': recall_score(y_true, y_pred, average='macro'),
        'F1-Score (Macro)': f1_score(y_true, y_pred, average='macro'),
        'Precision (Micro)': precision_score(y_true, y_pred, average='micro'),
        'Recall (Micro)': recall_score(y_true, y_pred, average='micro'),
        'F1-Score (Micro)': f1_score(y_true, y_pred, average='micro')
    }
    return metrics

## **Basic CNN** 

In [ ]:
def build_basic_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Create the model
basic_cnn_model = build_basic_cnn(input_shape=(9, 7, 1), num_classes=y_train.shape[1])

# Compile the model
basic_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = basic_cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Save the model
basic_cnn_model.save("mediapipe_basic_cnn.h5")


In [ ]:
basic_cnn_model.summary()

In [ ]:
# Predict on the test set
y_pred = basic_cnn_model.predict(X_test)

# Compute metrics (Macro and Micro)
metrics = compute_metrics(y_test, y_pred)

# Display metrics
print("\n--- Basic CNN Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
plot_training_history(history, title='Basic CNN Training Metrics')

## **Intermediate CNN**

In [ ]:
def build_intermediate_cnn(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Create the model
intermediate_cnn_model = build_intermediate_cnn(input_shape=(9, 7, 1), num_classes=y_train.shape[1])

# Compile the model
intermediate_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = intermediate_cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Save the model
intermediate_cnn_model.save("mediapipe_intermediate_cnn.h5")


In [ ]:
intermediate_cnn_model.summary()

In [ ]:
# Predict on the test set
y_pred = intermediate_cnn_model.predict(X_test)

# Compute metrics (Macro and Micro)
metrics = compute_metrics(y_test, y_pred)

# Display metrics
print("\n--- Intermediate CNN Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
plot_training_history(history, title='Intermediate CNN Training Metrics')

## **Advanced CNN**

In [ ]:
def build_advanced_cnn(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 1)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
# Create the model
advanced_cnn_model = build_advanced_cnn(input_shape=(9, 7, 1), num_classes=y_train.shape[1])

# Compile the model
advanced_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = advanced_cnn_model.fit(X_train, y_train, 
                                           validation_data=(X_test, y_test), 
                                           epochs=15, 
                                           batch_size=32)

# Save the model
advanced_cnn_model.save("mediapipe_advanced_cnn.h5")


In [ ]:
advanced_cnn_model.summary()

In [ ]:
# Predict on the test set
y_pred = advanced_cnn_model.predict(X_test)

# Compute metrics (Macro and Micro)
metrics = compute_metrics(y_test, y_pred)

# Display metrics
print("\n--- Advanced CNN Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
plot_training_history(history, title='Advanced CNN Training Metrics')

## **Residual CNN**

In [ ]:
def build_residual_cnn(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # First Convolutional Block
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    # Residual Block
    res = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    res = tf.keras.layers.BatchNormalization()(res)
    res = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(res)
    res = tf.keras.layers.BatchNormalization()(res)

    # Align dimensions using a 1x1 convolution
    x = tf.keras.layers.Conv2D(64, (1, 1), padding='same')(x)  # Adjust channels to 64
    x = tf.keras.layers.Add()([x, res])  # Skip connection
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    # Flatten and Dense Layers
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [ ]:
# Create the model
residual_cnn_model = build_residual_cnn(input_shape=(9, 7, 1), num_classes=y_train.shape[1])

# Compile the model
residual_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = residual_cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=13, batch_size=32)

# Save the model
residual_cnn_model.save("mediapipe_residual_cnn.h5")


In [ ]:
residual_cnn_model.summary()

In [ ]:
# Predict on the test set
y_pred = residual_cnn_model.predict(X_test)

# Compute metrics (Macro and Micro)
metrics = compute_metrics(y_test, y_pred)

# Display metrics
print("\n--- Residual CNN Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
plot_training_history(history, title='Residual CNN Training Metrics')

## **Deeper CNN**

In [ ]:
def build_deeper_cnn(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Create the model
deeper_cnn_model = build_deeper_cnn(input_shape=(9, 7, 1), num_classes=y_train.shape[1])

# Compile the model
deeper_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = deeper_cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Save the model
deeper_cnn_model.save("mediapipe_deeper_cnn.h5")


In [ ]:
deeper_cnn_model.summary()

In [ ]:
# Predict on the test set
y_pred = deeper_cnn_model.predict(X_test)

# Compute metrics (Macro and Micro)
metrics = compute_metrics(y_test, y_pred)

# Display metrics
print("\n--- Deeper CNN Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
plot_training_history(history, title='Deeper CNN Training Metrics')

## **Wide CNN**

In [ ]:
def build_wide_cnn(input_shape, num_classes):
    model = tf.keras.models.Sequential([

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2), padding='same'),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 1), padding='same'),

        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 1), padding='same'),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
# Create the model
wide_cnn_model = build_wide_cnn(input_shape=(9, 7, 1), num_classes=y_train.shape[1])

# Compile the model
wide_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = wide_cnn_model.fit(X_train, y_train, 
                                       validation_data=(X_test, y_test), 
                                       epochs=15, 
                                       batch_size=32)

# Save the model
wide_cnn_model.save("mediapipe_wide_cnn.h5")


In [ ]:
wide_cnn_model.summary()

In [ ]:
# Predict on the test set
y_pred = wide_cnn_model.predict(X_test)

# Compute metrics (Macro and Micro)
metrics = compute_metrics(y_test, y_pred)

# Display metrics
print("\n--- Wide CNN Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
plot_training_history(history, title='Wide CNN Training Metrics')

## **All Results Combined**

In [ ]:
# Initialize a list to store metrics for each model
metrics_list = []

# Function to evaluate a model and save metrics
def evaluate_and_save_metrics(model_name, model, X_test, y_test):
    """
    Evaluate the model on the test set and save the metrics to the metrics list.
    Args:
        model_name (str): Name of the model being evaluated.
        model (tf.keras.Model): The trained model.
        X_test (np.array): Test data (features).
        y_test (np.array): Test labels (one-hot encoded).
    """
    # Predict on the test set
    y_pred = model.predict(X_test)

    # Compute metrics
    metrics = compute_metrics(y_test, y_pred)

    # Add model name to metrics
    metrics['Model'] = model_name

    # Append metrics to the list
    metrics_list.append(metrics)


In [ ]:
# Evaluate models and save metrics
evaluate_and_save_metrics("Basic CNN", basic_cnn_model, X_test, y_test)
evaluate_and_save_metrics("Intermediate CNN", intermediate_cnn_model, X_test, y_test)
evaluate_and_save_metrics("Advanced CNN", advanced_cnn_model, X_test, y_test)
evaluate_and_save_metrics("Residual CNN", residual_cnn_model, X_test, y_test)
evaluate_and_save_metrics("Deeper CNN", deeper_cnn_model, X_test, y_test)
evaluate_and_save_metrics("Wide CNN", wide_cnn_model, X_test, y_test)

In [ ]:
from IPython.display import display

def display_metrics_table(metrics_list):
    """
    Display a visually enhanced table of metrics.
    Highlights max values in green and min values in red for each column.
    Args:
        metrics_list (list): List of dictionaries containing model metrics.
    """
    # Create a DataFrame from the metrics list
    metrics_df = pd.DataFrame(metrics_list)
    
    # Reorder columns for better readability
    metrics_df = metrics_df[["Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", 
                             "F1-Score (Macro)", "Precision (Micro)", "Recall (Micro)", "F1-Score (Micro)"]]
    
    # Define custom highlighting functions
    def highlight_max(s):
        is_max = s == s.max()
        return ['background-color: green; color: white' if v else '' for v in is_max]

    def highlight_min(s):
        is_min = s == s.min()
        return ['background-color: red; color: white' if v else '' for v in is_min]

    # Apply the custom styles to the DataFrame
    styled_table = (
        metrics_df.style
        .apply(highlight_max, subset=["Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)", 
                                      "Precision (Micro)", "Recall (Micro)", "F1-Score (Micro)"])
        .apply(highlight_min, subset=["Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)", 
                                      "Precision (Micro)", "Recall (Micro)", "F1-Score (Micro)"])
        .format(precision=4)  # Limit float values to 4 decimal points
        .set_caption("Model Evaluation Metrics Table")
        .set_table_styles(
            [
                {"selector": "caption", "props": [("text-align", "center"), ("font-size", "16px"), ("font-weight", "bold")]},
                {"selector": "thead th", "props": [("background-color", "#f4f4f4"), ("font-size", "14px"), ("font-weight", "bold")]},
                {"selector": "tbody td", "props": [("font-size", "12px"), ("text-align", "center")]},
            ]
        )
    )
    
    # Display the styled table
    display(styled_table)

# Call the function to display the table
display_metrics_table(metrics_list)

